In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
import numpy
a = numpy.random.randn(4,4)

In [3]:
a = a.astype(numpy.float32)

In [4]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [5]:
cuda.memcpy_htod(a_gpu, a)

In [6]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [7]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [8]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print (a_doubled)
print (a)

[[ 0.17743494 -0.10634408  2.4602919  -0.34212252]
 [-0.17244455 -1.9942837   3.1718068   4.1414213 ]
 [ 0.02055212 -0.3688605   0.201582    1.3475362 ]
 [ 2.5867765   1.4415287   2.7623808  -2.0446472 ]]
[[ 0.08871747 -0.05317204  1.2301459  -0.17106126]
 [-0.08622228 -0.99714184  1.5859034   2.0707107 ]
 [ 0.01027606 -0.18443026  0.100791    0.6737681 ]
 [ 1.2933882   0.72076434  1.3811904  -1.0223236 ]]
